In [1]:
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle

In [2]:
# init dict
Data = collections.defaultdict(list)

# Wikipedia

In [3]:
# URL for database
#URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
#URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_the_Netherlands'
URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_Europe'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
Animals = soup.find_all('a')
# Create links 
AnimalsWikiPages = ['https://en.wikipedia.org/' + pages.get('href') for pages in Animals 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]
                       # Reduces the retrieved pages (does not work)
                       #if pages.span != None 
                       #if pages.span.attrs['class'][0] == 'tocnumber']

In [4]:
# init dicts
DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiPage in AnimalsWikiPages[0:50]:
    
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Skip useless/empty stuff
        if Tags.span == None:
            continue

        # Set chapter variable    
        Chapter = Tags.span.attrs['id']

        # Check if the chapter is description (or similar)
        if Chapter == 'Characteristics'or \
           Chapter == 'Description' or \
           Chapter == 'Appearance':


            # Get species name
            Species = soup.title\
                            .string\
                            .split(' - ')[0]\
                            .rstrip(' ')


            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(1, Sentence) for Sentence in SentenceList]
                    #### TESTING
                    for index, Sentences in enumerate(SentenceList):
                        if all(s.lower() not in Sentences for s in ['cm', 'm', 'kg', 'lbs', 'dimensions']):
                            Data[Species].append(tuple([0, Sentences]))
                        else:
                            Data[Species].append(tuple([1, Sentences]))

                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    Data[Species] += [(0, Sentence) for Sentence in SentenceList]

# Animals A-Z

In [5]:
URL = 'https://a-z-animals.com/animals/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Finds all links
Animals = soup.find_all('a')

# Create a list with links
AnimalsA_Zpages = [pages.get('href') for pages in Animals 
                   if pages.get('href') != None
                   if pages.get('href').startswith('https://a-z-animals.com')]

In [6]:
# Removes references in text
ReferenceRemover = '\[\d*\]'

for AnimalPage in AnimalsA_Zpages[:]:
    
    # Open the page
    page = requests.get(AnimalPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Get the chapters
        try:
            Chapter = Tags['id']
        # skip other stuff
        except:
            continue

        # Finds descriptions or similar
        if Chapter.endswith('appearance'):
            
            '''
            # Get species name
            Species = soup.title\
                            .string\
                            .split(' Animal')[0]\
                            .lower()\
                            .capitalize()
            '''
            
            # Get species name
            Species = soup.find('h1').text\
                                        .lower()\
                                        .capitalize()


            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(1, Sentence) for Sentence in SentenceList]
                    #### TESTING
                    for index, Sentences in enumerate(SentenceList):
                        if all(s.lower() not in Sentences for s in ['cm', 'm', 'kg', 'lbs', 'dimensions']):
                            Data[Species].append(tuple([0, Sentences]))
                        else:
                            Data[Species].append(tuple([1, Sentences]))
  

                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    Data[Species] += [(0, Sentence) for Sentence in SentenceList]
        

In [7]:
with open('data.pkl', 'wb') as f:
    pickle.dump(Data, f)

# With Measurements

In [12]:
# init dicts
DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiPage in AnimalsWikiPages[0:50]:
    
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Skip useless/empty stuff
        if Tags.span == None:
            continue

        # Set chapter variable    
        Chapter = Tags.span.attrs['id']

        # Check if the chapter is description (or similar)
        if Chapter == 'Characteristics'or \
           Chapter == 'Description' or \
           Chapter == 'Appearance':


            # Get species name
            Species = soup.title\
                            .string\
                            .split(' - ')[0]\
                            .rstrip(' ')


            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Add to dict
                    Data[Species].append(tuple([1, Paragraph]))
                    
                    # Split into Sentences
                    #SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(1, Sentence) for Sentence in SentenceList]

                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Add to dict
                    Data[Species].append(tuple([0, Paragraph]))
                    
                    # Split into Sentences
                    #SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(0, Sentence) for Sentence in SentenceList]

In [13]:
# Removes references in text
ReferenceRemover = '\[\d*\]'

for AnimalPage in AnimalsA_Zpages[:]:
    
    # Open the page
    page = requests.get(AnimalPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Get the chapters
        try:
            Chapter = Tags['id']
        # skip other stuff
        except:
            continue

        # Finds descriptions or similar
        if Chapter.endswith('appearance'):
            
            '''
            # Get species name
            Species = soup.title\
                            .string\
                            .split(' Animal')[0]\
                            .lower()\
                            .capitalize()
            '''
            
            # Get species name
            Species = soup.find('h1').text\
                                        .lower()\
                                        .capitalize()


            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Add to dict
                    Data[Species].append(tuple([1, Paragraph]))
                    
                    # Split into Sentences
                    #SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(1, Sentence) for Sentence in SentenceList]
          
                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Add to dict
                    Data[Species].append(tuple([0, Paragraph]))
                    
                    # Split into Sentences
                    #SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(0, Sentence) for Sentence in SentenceList]
        

In [14]:
with open('data_withMeasurements.pkl', 'wb') as f:
    pickle.dump(Data, f)